In [1]:
import pandas as pd
import torch
import ast
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

c:\Users\hari_\anaconda3\envs\hbqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# @title Load Configuraiton 
import hbqaconfig, Utilfunc
kaggle=False
conf = hbqaconfig.setEnv('local')

In [3]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

DEVICE

device(type='cpu')

# Select & Load Embedding Model

In [5]:
embmodelname, embmodelshort, embmodelname1 = Utilfunc.select_embmodel(3)
embmodel = SentenceTransformer(embmodelname)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


# Load New Chunks and QA Dataset in Dataframe

In [6]:
df_chunk = pd.read_excel(conf['RAAGS_FOLDER'] + '10.1-HBQA-for-RAAGS.xlsx', sheet_name="Chunk")
df_qa = pd.read_excel(conf['RAAGS_FOLDER'] + '10.1-HBQA-for-RAAGS.xlsx', sheet_name="QA")

In [7]:
df_chunk.shape, df_qa.shape

((50, 2), (163, 4))

In [8]:
df_chunk.head(3)

,Chunk_Id,Chunk
0,1001,"This work is a tree, of which the chapter of c..."
1,1002,"This work is a tree, of which the chapter of c..."
2,1003,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...


In [9]:
df_qa.head(3)

,Ques_Id,Chunk_Id,Question,Ref_Answer
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D..."
1,2002,1001,"Narada recited the Mahabharata to the Devas, D...","The two sons of Madri, the twin Aswins, are co..."
2,2003,1001,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...


# Create New Chunk's and DA Dataset's Vector

## Chunk vector

In [11]:
# When some function is decorated with this function. It will show the time to execute that function.
# we will use this calculate the time for embedding.
def get_time(func):
    import time
    def wrapper():
        start_time = time.time()

        result = func()

        end_time = time.time()
        time_spent=end_time-start_time

        print(f"Time Taken to Process {time_spent}")
        return result

    return wrapper

In [12]:
# Convert Chunk colum values to List.
@get_time
def get_chunk_vector():
  Chunk_Sentences = df_chunk.Chunk.tolist()
  Chunk_Embeddings = embmodel.encode(Chunk_Sentences)
  return Chunk_Embeddings

Chunk_Embeddings = get_chunk_vector()

# Convert These embedding into torch tensor and load into available DEVICE
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float).to(DEVICE)

# Flatten the tensors into 1D arrays/list
# Create ChunkVector in dataset.'
df_chunk['ChunkVector'+embmodelname1 ] = ChunkVectors.tolist()



Time Taken to Process 90.84815168380737


In [13]:
print(df_chunk.shape)
df_chunk.head(3)

(50, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,1001,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
1,1002,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
2,1003,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,"[-0.0005072467029094696, -0.08607526123523712,..."


In [14]:
# Save the Vector on harddisk.
filenm = "10.2-HBQA_RAAGS_chunk_Vector" + embmodelname1 +".csv"
df_chunk.to_csv(conf['RAAGS_FOLDER'] + filenm, index=None)

## QA Vector

In [15]:
# Convert Chunk colum values to List.
@get_time
def get_qa_vector():
  Question_Sentences = df_qa.Question.tolist()
  Ref_Answer_Sentences = df_qa.Ref_Answer.tolist()
  Question_Embeddings = embmodel.encode(Question_Sentences)
  Ref_Answer_Embeddings = embmodel.encode(Ref_Answer_Sentences)
  return Question_Embeddings, Ref_Answer_Embeddings

Question_Embeddings, Ref_Answer_Embeddings = get_qa_vector()

Time Taken to Process 40.692105293273926


In [16]:
# Convert These embedding into torch tensor and load into available DEVICE
QuestionVectors= torch.tensor(Question_Embeddings, dtype=torch.float).to(DEVICE)
Ref_AnswerVectors= torch.tensor(Ref_Answer_Embeddings, dtype=torch.float).to(DEVICE)

# Flatten the tensors into 1D arrays/list
# Create ChunkVector in dataset.'
df_qa['QuesVector'+embmodelname1 ] = QuestionVectors.tolist()
df_qa['AnsVector'+embmodelname1 ] = Ref_AnswerVectors.tolist()

print(embmodelname)
	
df_qa.head(3)

multi-qa-mpnet-base-dot-v1


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuestionVector_multi-qa-mpnet-base-dot-v1,Ref_AnswerVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,"Narada recited the Mahabharata to the Devas, D...","The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."
2,2003,1001,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...,"[0.18756987154483795, -0.16670864820480347, -0...","[0.01948237046599388, -0.008955039083957672, -..."


In [17]:
# Save the Vector on harddisk.
filenm = "10.3-HBQA_RAAGS_QA_Vector" + embmodelname1 +".csv"
df_qa.to_csv(conf['RAAGS_FOLDER'] + filenm, index=None)

# Load New Chunks and QA Vector Dataset from Harddisk

In [64]:
filenm = "10.2-HBQA_RAAGS_chunk_Vector" + embmodelname1 +".csv"
df_chunk1 = pd.read_csv(conf['RAAGS_FOLDER'] + filenm)

filenm = "10.3-HBQA_RAAGS_QA_Vector" + embmodelname1 +".csv"
df_qa1 = pd.read_csv(conf['RAAGS_FOLDER'] + filenm)

# Load Chunks and Question 
(QAS System is trained on this)

In [65]:
print(df_qa1.shape)
df_qa1.head(2)

(163, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."


In [66]:
filenm= "07.2-HBQA_QA_Vector" + embmodelname1 +".csv"
print(filenm)
df_qa2 = pd.read_csv(conf['QAGS_FOLDER'] + filenm,\
                      usecols= ['Ques_Id', 'Chunk_Id', 'Question', 'Ref_Answer', \
                                'QuesVector' + embmodelname1, 'AnsVector' + embmodelname1])
# df_qa2 = df_qa2.set_index(df_qa2.Ques_Id)
print(df_qa2.shape)
df_qa2.head(2)

07.2-HBQA_QA_Vector_multi-qa-mpnet-base-dot-v1.csv
(1102, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,0,389,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,"[-0.4276844561100006, 0.07686357200145721, -0....","[-0.316677063703537, 0.04548855870962143, -0.2..."
1,1,390,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"[-0.04931388795375824, -0.15976008772850037, -...","[-0.013333199545741081, 0.05390685796737671, -..."


In [67]:
print(df_chunk1.shape)
df_chunk1.head(2)

(50, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,1001,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
1,1002,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."


In [69]:
filenm= '04.3-HBQA_Chunk_Vector'+embmodelname1+'.csv'
print(filenm)
df_chunk2 = pd.read_csv(conf['PE_FOLDER'] + filenm, usecols=['Chunk_Id','Chunk','ChunkVector'+embmodelname1])
print(df_chunk2.shape)
df_chunk2.head(2)

04.3-HBQA_Chunk_Vector_multi-qa-mpnet-base-dot-v1.csv
(334, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,389,Even this is eternal morality. They that perfo...,"[-0.044648099690675735, -0.0822770893573761, -..."
1,390,"Section III\n""Vaisampayana said, 'Yudhishthira...","[0.2307928055524826, -0.18484173715114594, -0...."


# Merge Chunks and QA Vector Datasets

In [72]:
# Concat Chunk and Reindex
df_chunk = pd.concat([df_chunk1, df_chunk2])
# df_chunk = df_chunk2.copy() 

df_chunk.index = range(df_chunk.shape[0])
print(df_chunk.shape)
df_chunk.head(3)

(384, 3)


,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
0,1001,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
1,1002,"This work is a tree, of which the chapter of c...","[0.23210011422634125, -0.17947782576084137, -0..."
2,1003,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,"[-0.0005072467029094696, -0.08607526123523712,..."


In [74]:
df_chunk.tail(3)

,Chunk_Id,Chunk,ChunkVector_multi-qa-mpnet-base-dot-v1
381,961,"Section LXXXI\n""Sahadeva said, 'What hath been...","[0.15935538709163666, -0.3883747458457947, -0...."
382,974,"Thus consoled by Krishna, Kunti, afflicted wit...","[0.2113616168498993, -0.1391368806362152, -0.1..."
383,989,"Section CI\n""Narada said, 'This region belonge...","[0.08386620879173279, 0.02076539397239685, -0...."


In [75]:
# Concat Question and Reindex 
df_qa = pd.concat([df_qa1, df_qa2])
# df_qa = df_qa2.copy() 

df_qa.index = range(df_qa.shape[0])
print(df_qa.shape)
df_qa.head(3)

(1265, 6)


,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
0,2001,1001,"Who recited the Mahabharata to the Devas, Pitr...","Narada recited the Mahabharata to the Devas, D...","[-0.1249421238899231, -0.025616921484470367, -...","[0.033066216856241226, -0.24593223631381989, -..."
1,2002,1001,Who are compared to the fruit and flowers of t...,"The two sons of Madri, the twin Aswins, are co...","[0.03306629881262779, -0.24593225121498108, -0...","[-0.15711747109889984, -0.3908252716064453, -0..."
2,2003,1001,How did the citizens react when the young prin...,The distinguished class of citizens shouted fo...,"[0.18756987154483795, -0.16670864820480347, -0...","[0.01948237046599388, -0.008955039083957672, -..."


In [76]:
df_qa.tail(3)

,Ques_Id,Chunk_Id,Question,Ref_Answer,QuesVector_multi-qa-mpnet-base-dot-v1,AnsVector_multi-qa-mpnet-base-dot-v1
1262,1099,961,What is Sahadeva's desire and the reason behin...,Sahadeva desires to provoke a war with the Kau...,"[0.1654556393623352, -0.40023961663246155, -0....","[0.02510783076286316, -0.6113841533660889, -0...."
1263,1100,974,How did Kunti respond to Krishna's counsel reg...,Kunti responded by placing her full trust in K...,"[0.29927170276641846, 0.0765724703669548, -0.2...","[0.31003451347351074, -0.05984937772154808, -0..."
1264,1101,989,Can you provide the names of some prominent ch...,Narada listed several prominent chiefs of the ...,"[0.07110704481601715, -0.06087541580200195, -0...","[0.12445423007011414, -0.29634472727775574, -0..."


# Predict Chunk for Question

In [77]:
# Load all chunk_vectors into memotry
chunk_id = df_chunk['Chunk_Id'].to_list()

chunk_vector = [ast.literal_eval(df_chunk['ChunkVector'+embmodelname1][i]) for i in range(df_chunk.shape[0])]
# chunk_vector = [df_chunk['ChunkVector'+embmodelname1][i] for i in range(df_chunk.shape[0])]
chunk_vector = torch.tensor(chunk_vector)

In [78]:
# Search Chunk based on Each Question. Get 5 chunk for each question
df_chunk_pred = pd.DataFrame(columns = ['Ques_Id','Chunk_Id','Pred_Chunk_Id'])

for i in df_qa.index:
  question_embedding = ast.literal_eval( df_qa.loc[ i,'QuesVector' + embmodelname1])  
  question_embedding = torch.tensor(question_embedding)

  # print (type(question_embedding), question_embedding)
  top_k=5
  hits = util.semantic_search(question_embedding, chunk_vector, top_k=top_k)


  predictedId= [item['corpus_id'] for item in hits[0]] # Hit contains index (corpus_id), location of chunk_vector
  predicted_Docid = [df_chunk.loc[i, "Chunk_Id"] for i in predictedId] # We need to get chunk_id corresponding to chunk_index
  # print (predicted_Docid)
  df_chunk_pred.loc[i]= (df_qa.loc[i, 'Ques_Id'], df_qa.loc[i,'Chunk_Id'], predicted_Docid) # save predictions
  print('Predicting: ',i)
  # break

Predicting:  0
Predicting:  1
Predicting:  2
Predicting:  3
Predicting:  4
Predicting:  5
Predicting:  6
Predicting:  7
Predicting:  8
Predicting:  9
Predicting:  10
Predicting:  11
Predicting:  12
Predicting:  13
Predicting:  14
Predicting:  15
Predicting:  16
Predicting:  17
Predicting:  18
Predicting:  19
Predicting:  20
Predicting:  21
Predicting:  22
Predicting:  23
Predicting:  24
Predicting:  25
Predicting:  26
Predicting:  27
Predicting:  28
Predicting:  29
Predicting:  30
Predicting:  31
Predicting:  32
Predicting:  33
Predicting:  34
Predicting:  35
Predicting:  36
Predicting:  37
Predicting:  38
Predicting:  39
Predicting:  40
Predicting:  41
Predicting:  42
Predicting:  43
Predicting:  44
Predicting:  45
Predicting:  46
Predicting:  47
Predicting:  48
Predicting:  49
Predicting:  50
Predicting:  51
Predicting:  52
Predicting:  53
Predicting:  54
Predicting:  55
Predicting:  56
Predicting:  57
Predicting:  58
Predicting:  59
Predicting:  60
Predicting:  61
Predicting:  62
Pr

In [17]:
tmpfilenm = "10.1-Predict-DocumentId-for-Ques"+embmodelname1+".csv"
df_chunk_pred.to_csv( conf['RAAGS_FOLDER'] + tmpfilenm,index=None)

In [18]:
# Mean Reciprocal Rank (MRR) is a metric commonly used to evaluate the performance of ranking-based retrieval systems,
# such as search engines. MRR assesses how well a retrieval system ranks relevant items by considering the position of the
# first relevant item in the ranked list. It's calculated as the average of the reciprocal ranks across a set of queries.

def getReciprocalRank(row):
  K=1 # Number of relevent document
  try:   
    i = row['Pred_Chunk_Id'].index( row['Chunk_Id']) #ast.literal_eval( row['Pred_Chunk_Id']).index(row['Chunk_Id'] )
  except:
    i=-1

  if i==-1:
    RR=0
  else:
    RR = round(1/(i + 1),2)

#   print (i)
  return RR

df_chunk_pred['RR'] = df_chunk_pred.apply(getReciprocalRank, axis=1)

In [19]:
df_chunk_pred.head(10)

,Ques_Id,Chunk_Id,Pred_Chunk_Id,RR
0,2001,1001,"[76, 1003, 693, 690, 593]",0.00
1,2002,1001,"[76, 704, 1004, 602, 703]",0.00
2,2003,1001,"[1006, 212, 811, 395, 707]",0.00
3,2004,1001,"[235, 345, 477, 453, 717]",0.00
4,2005,1002,"[1001, 1002, 76, 1005, 499]",0.50
5,2006,1002,"[1001, 1002, 1010, 596, 1005]",0.50
6,2007,1002,"[557, 452, 578, 593, 693]",0.00
7,2008,1002,"[1006, 212, 712, 702, 155]",0.00
8,2009,1003,"[605, 76, 702, 1003, 583]",0.25
9,2010,1003,"[76, 1004, 1003, 1010, 645]",0.33


In [20]:
df_chunk_pred['RR'].mean()

0.5424426877470356

In [21]:
df_chunk_pred.sort_values('RR',ascending=False)

,Ques_Id,Chunk_Id,Pred_Chunk_Id,RR
1264,1101,989,"[989, 1022, 69, 469, 1034]",1.0
810,647,655,"[655, 662, 661, 660, 657]",1.0
392,229,478,"[478, 669, 696, 433, 477]",1.0
393,230,478,"[478, 470, 477, 480, 735]",1.0
394,231,478,"[478, 717, 477, 480, 712]",1.0
...,...,...,...,...
637,474,556,"[395, 235, 783, 393, 398]",0.0
1,2002,1001,"[76, 704, 1004, 602, 703]",0.0
631,468,555,"[502, 515, 520, 556, 553]",0.0
629,466,555,"[515, 556, 520, 502, 583]",0.0


# Select & Load Prediction Model

## Select Model

## Load Model

# Get 5 answer from top 5 document for each question

In [79]:
#Get 5 answer from top 5 document for each question
tmpfilenm = "10.1-Predict-DocumentId-for-Ques"+embmodelname1+".csv"
df_chunk_pred = pd.read_csv( conf['RAAGS_FOLDER'] + tmpfilenm)

In [80]:
for i in range(df_chunk_pred.shape[0]):
    print ("Question Id",df_chunk_pred.loc[i,'Ques_Id'])
    ids = df_chunk_pred.loc[i,"Pred_Chunk_Id"]
    ques_id = df_chunk_pred.loc[i,"Ques_Id"]

    cond = df_qa["Ques_Id"] == ques_id 
    question = df_qa[cond]["Question"].values[0]
    print(question)

    if len(ids)>3:
        ids = ast.literal_eval(ids)
    else:
        print("Sorry, No document")
        break

    for id in ids:
        # print(ques_id, id)
        cond = df_chunk["Chunk_Id"]==id
        chunk = df_chunk.loc[cond]["Chunk"].values[0]

        print(chunk[:20])

Question Id 2001
Who recited the Mahabharata to the Devas, Pitris, Gandharvas, and mankind?
Section LIX
(Adivans
THE MAHABHARATA ADI 
Section CCXLII
Vaisa
Section CCXXXIX
Vais
Section CLXVII
"Arju
Question Id 2002
Who are compared to the fruit and flowers of the vast tree formed by Yudhishthira? 
Section LIX
(Adivans
Section CCLIII
Vaisa
And what is seen in 
Section CLXXIV
Vaisa
Section CCLII
Vaisam
Question Id 2003
How did the citizens react when the young princes, the Pandavas, were introduced as the sons of Pandu?
Pandu, after having 
Section CLXVII
(Chai
Section XXIX
"Vaisam
Section VII
"Vaisamp
Section CCLVI
Janame
Question Id 2004
Who obtained the virgin Krishna at the swayamvara, and what did he accomplish afterward?
Section CLXXXVIII
(S
"After all the Kings
Section LXXIV
"Damay
Section LIV
"Vrihada
Section CCLXV
Vaisam
Question Id 2005
What is the significance of the book named Mausala in the text?
This work is a tree,
This work is a tree,
Section LIX
(Adivans
Ganesa, upon hear

In [ ]:
# Load t5_30epoch finetuned model. 
# Use Summarise Chunk and Question to Find Answer Using AGS
